In [12]:
from VoxResNetPytorch import *
import pandas as pd
import numpy as np
import os
import nibabel as nib

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Function
from torch.autograd import gradcheck

import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torchvision.datasets.folder import *



In [13]:
def load_nii_to_array(nii_path):
    return nib.load(nii_path).get_data()

In [14]:
def preproc_data(data_file,return_y = True):
    #Encode with 1, psycho, and with 0 normal
    data_file['target'] = 1
    data_file.target[data_file['diagnosis'] == 'CONTROL'] = 0 
    if return_y == True:
        y = participants_pandas[['participant_id','target']]
        return y
    return data_file

In [15]:
class Base_loader(data.Dataset):
    """ Some documantation"""

    def __init__(self,labels, root_dir, transform=None,loader=default_loader):
        """
        Args:
        """
       
        #First file is targets
        self.root = os.listdir(root_dir)[1:]
        self.root_path = root_dir
        #self.files = imgs
        #self.imgs = 
        self.classes = labels.target
        self.class_to_idx = labels.participant_id
        self.transform = transform
        self.loader = loader


    def __getitem__(self, idx):
        path, target = self.root[idx], self.classes[idx]
        path = os.path.join(self.root_path,path)
        img = load_nii_to_array(path)
        if self.transform is not None:
            img = self.transform(img)
            
        return img, target
        
    
    def __len__(self):
        return len(self.imgs)

In [16]:
data_folder = 'data/'
targets_file ='LA5study_targets.csv' 
participant_info = data_folder + targets_file
participants_pandas = pd.read_csv(participant_info)
y = preproc_data(participants_pandas)
participants_pandas.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,participant_id,diagnosis,age,gender,bart,bht,dwi,pamenc,pamret,rest,...,T1w,taskswitch,ScannerSerialNumber,ghost_NoGhost,label,Bipolar/Control,Bipolar/NotBipolar,ADHD/Control,Schz/Control,target
0,sub-10159,CONTROL,30,F,1.0,NaN,1.0,NaN,NaN,1.0,...,1.0,1.0,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0
1,sub-10171,CONTROL,24,M,1.0,1.0,1.0,NaN,NaN,1.0,...,1.0,1.0,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0
2,sub-10189,CONTROL,49,M,1.0,NaN,1.0,NaN,NaN,1.0,...,1.0,1.0,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0
3,sub-10193,CONTROL,40,M,1.0,NaN,1.0,NaN,NaN,NaN,...,1.0,NaN,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0
4,sub-10206,CONTROL,21,M,1.0,NaN,1.0,NaN,NaN,1.0,...,1.0,1.0,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0


In [17]:
loader = Base_loader(y,data_folder)
vox_net = VoxResNet()
vox_net


VoxResNet(
  (model): Sequential(
    (conv3d_1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (batch_norm_1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True)
    (activation_1): ReLU(inplace)
    (conv3d_2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (batch_norm_2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True)
    (activation_2): ReLU(inplace)
    (conv3d_3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (block_1): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (block_2): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3),